<a href="https://colab.research.google.com/github/ReyanPatel/PlacardIntel/blob/main/rpi_yolov11_object_detection_on_custom_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 import os
HOME = os.getcwd()
print(HOME)

/content


## Install YOLOv8

⚠️ YOLOv8 is still under heavy development. Breaking changes are being introduced almost weekly. We strive to make our YOLOv8 notebooks work with the latest version of the library. Last tests took place on **27.01.2023** with version **YOLOv8.0.20**.

If you notice that our notebook behaves incorrectly - especially if you experience errors that prevent you from going through the tutorial - don't hesitate! Let us know and open an [issue](https://github.com/roboflow/notebooks/issues) on the Roboflow Notebooks repository.

YOLOv8 can be installed in two ways - from the source and via pip. This is because it is the first iteration of YOLO to have an official package.

In [ ]:
# Pip install method (recommended)

!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.163 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.7/112.6 GB disk)


In [ ]:
# Git clone method (for development)

# %cd {HOME}
# !git clone github.com/ultralytics/ultralytics
# %cd {HOME}/ultralytics
# !pip install -qe ultralytics

# from IPython import display
# display.clear_output()

# import ultralytics
# ultralytics.checks()

In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

## CLI Basics

If you want to train, validate or run inference on models and don't need to make any modifications to the code, using YOLO command line interface is the easiest way to get started. Read more about CLI in [Ultralytics YOLO Docs](https://v8docs.ultralytics.com/cli/).

```
yolo task=detect    mode=train    model=yolov8n.yaml      args...
          classify       predict        yolov8n-cls.yaml  args...
          segment        val            yolov8n-seg.yaml  args...
                         export         yolov8n.pt        format=onnx  args...
```

## Inference with Pre-trained COCO Model

### 💻 CLI

`yolo mode=predict` runs YOLOv8 inference on a variety of sources, downloading models automatically from the latest YOLOv8 release, and saving results to `runs/predict`.

In [ ]:
%cd {HOME}
!yolo task=detect mode=predict model=yolo11n.pt conf=0.25 source='https://media.roboflow.com/notebooks/examples/dog.jpeg' save=True

/content
100% 5.35M/5.35M [00:00<00:00, 88.1MB/s]
Ultralytics 8.3.163 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs

100% 104k/104k [00:00<00:00, 136MB/s]
image 1/1 /content/dog.jpeg: 640x384 2 persons, 1 car, 1 dog, 1 handbag, 44.2ms
Speed: 13.1ms preprocess, 44.2ms inference, 366.4ms postprocess per image at shape (1, 3, 640, 384)
Results saved to runs/detect/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive



Mounted at /content/gdrive
 Accessibility.gslides
'Application Problem Unit 3.gdoc'
'Application Problem Unit 3.pdf'
'ATLAS Grade 7 Science Summative.pdf'
 Boy.gdoc
 Classroom
'Colab Notebooks'
'Copy of [TEMPLATE] Python for Data Science Capstone Presentation.gslides'
'Course For Nick.gdoc'
'disneyland rides.gdoc'
'EAST Project For Conference.gslides'
 freedomtech.zip
'Getting started.pdf'
'GH&L.gdoc'
'Greek Gods.gdoc'
 IMG_0292.png
 IMG_0334.jpg
'ISEF Ideas.gdoc'
'ISEF Project Timeline - Personal.gslides'
'ISEF Research.gslides'
'ISEF Research Guiding Topics.gdoc'
'Kapil Sharma Script.gdoc'
' Kindergartener.gdoc'
'LeadCodePassword and Username.gdoc'
 MaPr.gdoc
'NN (1).pdf'
'Project proposal.gdoc'
 Questons.gdoc
 Random.gslides
'Raspberry Pi and VNC.gdoc'
'Research Paper.gdoc'
'RhythmiQ Confedential App Design.gslides'
 RoboticPrototypeVideo.mov
'Schedule 2024.gdoc'
'Screen Video.gdoc'
 SmartPodVideo.mp4
'Spanish Test Prep..gdoc'
'Tennis Rackets For Coach - Reyan Patel.gdoc'
'Toast Mas

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

!unzip /content/gdrive/MyDrive/freedomtech.zip

mkdir: cannot create directory ‘/content/datasets’: File exists
/content/datasets
Archive:  /content/gdrive/MyDrive/freedomtech.zip
   creating: freedomtech/images/
   creating: freedomtech/images/validation/
  inflating: freedomtech/images/validation/images_original_2135148_Still-1742179873.jpg_dc4cbe1e-9247-4de3-9f96-f9824fc25cad.txt  
  inflating: freedomtech/images/validation/images_original_ren-DMV43-608346521.jpg_d7e00a30-1c55-40bb-bdd9-c3a096003e10.txt  
  inflating: freedomtech/images/validation/images_original_2135148_Still-1742179873.jpg_46874b97-ef35-491a-8279-9c288c2defec.jpg  
  inflating: freedomtech/images/validation/images_original_91Br4aQ+34L._AC_-831167986.jpg_dffbc6b5-b484-4945-94d9-63fee89cb9dc.jpg  
  inflating: freedomtech/images/validation/images_original_fae07469-b0e2-4853-8dba-468c0fc7a1ce.__CR0,0,1464,600_PT0_SX1464_V1___-3229871858.jpg_598013b2-777e-4d7d-989f-d80004288959.txt  
  inflating: freedomtech/images/validation/images_original_Ohio-Handicap-Parking-P

## Custom Training

In [ ]:
%cd {HOME}

!yolo task=detect mode=train model=yolo11n.pt data=/content/datasets/freedomtech/data.yaml epochs=500 imgsz=240 plots=True

/content
Ultralytics 8.3.163 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/datasets/freedomtech/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=500, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=240, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, 